Plot pentru senzorul de vibratie.

In [11]:
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import tkinter as tk
import numpy as np
import serial.tools.list_ports
import re
import serial as sr

#----- global variables ----
data = np.array([])
data_color = np.array([])
cond = False

       
        
#----- plot data ----
def plot_data():
    global cond,data # true= am apasat pe butonul de Start si atunci facem plot 
    
    if(cond == True):
        a = s.readline()
        a_dec = a.decode()
        val_senzor = re.findall(r'\d+',a_dec)
        
        if len(val_senzor)!=0:
            
            if(len(val_senzor[0])>1):
                val_senzor[0]=1
            
                      
            if(len(data)<100):
                data = np.append(data,float(val_senzor[0]))
                
            else:
                data[0:99] = data[1:100]
                data[99]=float(val_senzor[0])
                


            lines.set_xdata(np.arange(0,len(data)))
            lines.set_ydata(data)
          
            canvas.draw()
           
    root.after(1,plot_data)
    
    
def plot_start():
    global cond
    cond = True
    s.reset_input_buffer()
    
def plot_stop():
    global cond
    cond = False
    
#-----Main GUI code------
root = tk.Tk()
root.title('Real Time plot test')
root.configure(background = 'light blue')
root.geometry("900x500")

#-----create Plot object on GUI-------
fig  = Figure()
ax = fig.add_subplot(111)

ax.set_title('Digital Vibration Sensor Data')
ax.set_xlabel('Ultimele 100 de valori')
ax.set_ylabel('Vibratie')
ax.set_xlim(0,100)
ax.set_ylim(-0.5,1.5)

lines = ax.plot([],[])[0] 

canvas = FigureCanvasTkAgg(fig, master = root)
canvas.get_tk_widget().place(x=10,y=10,width=600,height=400)
canvas.draw()


#----- creare buton
root.update();
start = tk.Button(root, text="Start", font=('calibri',12),command=lambda:plot_start())
start.place (x=50, y =450)

root.update();
stop = tk.Button(root, text="Stop", font=('calibri',12),command=lambda:plot_stop())
stop.place(x= start.winfo_x()+start.winfo_reqwidth()+20, y=450)


# start serial port
s = sr.Serial('COM6',115200)
s.reset_input_buffer()

root.after(1,plot_data)
##
root.mainloop()